In [1]:
import pandas as pd
from sklearn import preprocessing
import statsmodels.api as sm
import numpy as np

In [2]:
df = pd.read_csv("output.csv")

In [3]:
# nastavení prediktorů
predictors = ["surface_code","pl1_hand_code","pl2_hand_code","round_code","court_code","series_code","best_of","pl1_rank","pl2_rank","pl1_pts","pl2_pts","pl1_year_pro","pl2_year_pro","pl1_weight","pl2_weight","pl1_height","pl2_height",'pl1_avg_bookmaker_odds','pl2_avg_bookmaker_odds','pl1_total_games','pl1_win_rate','pl2_total_games','pl2_win_rate','pl1_swrate','pl2_swrate','rank_diff','pts_diff','wrate_diff','swrate_diff']

# tvorba train a test vzorků dat
train = df[df["date"] < "2019-01-01"]
test = df[df["date"] >= "2019-01-01"]

# rozlišení prediktorů a závislé proměnné pro trénink a test dat
X_train, y_train = train[predictors], train['result']
X_test, y_test = test[predictors], test['result']

# standardizace dat - prevence přetrénování
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
# nastavení prediktorů
predictors = ["surface_code","pl1_hand_code","pl2_hand_code","round_code","court_code","series_code","best_of","pl1_rank","pl2_rank","pl1_pts","pl2_pts","pl1_year_pro","pl2_year_pro","pl1_weight","pl2_weight","pl1_height","pl2_height",'pl1_avg_bookmaker_odds','pl2_avg_bookmaker_odds','pl1_total_games','pl1_win_rate','pl2_total_games','pl2_win_rate','pl1_swrate','pl2_swrate','rank_diff','pts_diff','wrate_diff','swrate_diff']

df = pd.DataFrame(predictors, columns=['name'])

df.to_csv('predictors.csv', index=False)

In [5]:
# přidání konstanty
X_train_sm = sm.add_constant(X_train)

# nastavení modelu
logit_model = sm.Logit(y_train, X_train_sm)

# trénink modelu
result = logit_model.fit()

# výstup logistické regrese
print(result.summary())

# výpočet přesnosti modelu
X_test_sm = sm.add_constant(X_test)
pred_probs = result.predict(X_test_sm)
y_pred = np.where(pred_probs >= 0.5, 1, 0)
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)

Optimization terminated successfully.
         Current function value: 0.557809
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 result   No. Observations:                26404
Model:                          Logit   Df Residuals:                    26374
Method:                           MLE   Df Model:                           29
Date:                Thu, 06 Jun 2024   Pseudo R-squ.:                  0.1953
Time:                        23:22:04   Log-Likelihood:                -14728.
converged:                       True   LL-Null:                       -18302.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0029      0.016      0.179      0.858      -0.029       0.035
x1            -0.0059      0.

In [ ]:
# Přidání sloupců do datasetu test pro analýzu ziskovosti sázkových strategií
test['pl1_prob'] = pred_probs
test['pred_result'] = np.where(test['pl1_prob'] >= 0.5, 1, 0)
test['bookmaker_result'] = np.where(test['pl1_avg_bookmaker_odds'] < test['pl2_avg_bookmaker_odds'], 1, 0)

lr_output = test[['id','pl1','pl2','date','pl1_avg_bookmaker_odds','pl2_avg_bookmaker_odds','result','pl1_prob','pred_result','bookmaker_result']]

# Display the DataFrame with the new column
lr_output.to_excel('lr_output.xlsx')